In [ ]:
"""
Exploratory Data Analysis for Gold Price Prediction
==================================================

This notebook provides comprehensive exploratory data analysis
for Indonesian gold price data.
"""

# Cell 1: Import Libraries and Setup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("📊 Gold Price EDA - Indonesian Market Analysis")
print("=" * 50)

# Cell 2: Load and Inspect Data
# Load the sample data
import json
import sys
import os

# Add parent directory to path
sys.path.append('..')

# Load data
data_path = '../data/sample_data.json'
if os.path.exists(data_path):
    with open(data_path, 'r') as f:
        data = json.load(f)
    df = pd.DataFrame(data)
    print(f"✅ Data loaded successfully: {len(df)} records")
else:
    print("❌ Sample data not found. Creating synthetic data...")
    # Create synthetic data for demonstration
    dates = pd.date_range('2024-01-01', periods=365, freq='D')
    base_price = 750000
    trend = np.linspace(0, 50000, 365)
    seasonal = 20000 * np.sin(2 * np.pi * np.arange(365) / 365)
    noise = np.random.normal(0, 10000, 365)
    
    harga_jual = base_price + trend + seasonal + noise
    harga_beli = harga_jual - np.random.uniform(10000, 20000, 365)
    
    df = pd.DataFrame({
        'tanggal': dates.strftime('%Y-%m-%d'),
        'hargaJual': harga_jual,
        'hargaBeli': harga_beli
    })
    print(f"✅ Synthetic data created: {len(df)} records")

# Convert date column
df['tanggal'] = pd.to_datetime(df['tanggal'])
df = df.sort_values('tanggal').reset_index(drop=True)

# Display basic info
print(f"\n📈 Dataset Overview:")
print(f"Date range: {df['tanggal'].min().date()} to {df['tanggal'].max().date()}")
print(f"Total days: {len(df)}")
print(f"\n💰 Price Statistics (IDR):")
print(f"Selling Price - Mean: {df['hargaJual'].mean():,.0f}, Std: {df['hargaJual'].std():,.0f}")
print(f"Buying Price - Mean: {df['hargaBeli'].mean():,.0f}, Std: {df['hargaBeli'].std():,.0f}")

# Cell 3: Data Quality Assessment
print("\n🔍 Data Quality Assessment")
print("=" * 30)

# Missing values
missing_data = df.isnull().sum()
print(f"Missing Values:\n{missing_data}")

# Data types
print(f"\nData Types:\n{df.dtypes}")

# Basic statistics
print(f"\nDescriptive Statistics:")
print(df.describe())

# Check for duplicates
duplicates = df.duplicated().sum()
print(f"\nDuplicate rows: {duplicates}")

# Check for negative values
negative_jual = (df['hargaJual'] < 0).sum()
negative_beli = (df['hargaBeli'] < 0).sum()
print(f"Negative prices - Jual: {negative_jual}, Beli: {negative_beli}")

# Cell 4: Price Distribution Analysis
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Gold Price Distribution Analysis', fontsize=16, fontweight='bold')

# Selling price histogram
axes[0, 0].hist(df['hargaJual'], bins=30, alpha=0.7, color='gold', edgecolor='black')
axes[0, 0].set_title('Selling Price Distribution')
axes[0, 0].set_xlabel('Price (IDR)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].axvline(df['hargaJual'].mean(), color='red', linestyle='--', label=f'Mean: {df["hargaJual"].mean():,.0f}')
axes[0, 0].legend()

# Buying price histogram
axes[0, 1].hist(df['hargaBeli'], bins=30, alpha=0.7, color='silver', edgecolor='black')
axes[0, 1].set_title('Buying Price Distribution')
axes[0, 1].set_xlabel('Price (IDR)')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].axvline(df['hargaBeli'].mean(), color='red', linestyle='--', label=f'Mean: {df["hargaBeli"].mean():,.0f}')
axes[0, 1].legend()

# Box plots
box_data = [df['hargaJual'], df['hargaBeli']]
bp = axes[1, 0].boxplot(box_data, labels=['Selling', 'Buying'], patch_artist=True)
bp['boxes'][0].set_facecolor('gold')
bp['boxes'][1].set_facecolor('silver')
axes[1, 0].set_title('Price Box Plots')
axes[1, 0].set_ylabel('Price (IDR)')

# Price spread analysis
df['spread'] = df['hargaJual'] - df['hargaBeli']
df['spread_pct'] = (df['spread'] / df['hargaBeli']) * 100

axes[1, 1].hist(df['spread'], bins=30, alpha=0.7, color='green', edgecolor='black')
axes[1, 1].set_title('Bid-Ask Spread Distribution')
axes[1, 1].set_xlabel('Spread (IDR)')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].axvline(df['spread'].mean(), color='red', linestyle='--', label=f'Mean: {df["spread"].mean():,.0f}')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

print(f"\n💹 Spread Analysis:")
print(f"Average spread: {df['spread'].mean():,.0f} IDR ({df['spread_pct'].mean():.2f}%)")
print(f"Spread range: {df['spread'].min():,.0f} - {df['spread'].max():,.0f} IDR")

# Cell 5: Time Series Analysis
fig, axes = plt.subplots(3, 1, figsize=(15, 12))
fig.suptitle('Gold Price Time Series Analysis', fontsize=16, fontweight='bold')

# Price trends
axes[0].plot(df['tanggal'], df['hargaJual'], label='Selling Price', color='gold', linewidth=2)
axes[0].plot(df['tanggal'], df['hargaBeli'], label='Buying Price', color='silver', linewidth=2)
axes[0].set_title('Gold Prices Over Time')
axes[0].set_ylabel('Price (IDR)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Price changes
df['price_change'] = df['hargaJual'].pct_change() * 100
axes[1].plot(df['tanggal'], df['price_change'], color='blue', alpha=0.7)
axes[1].axhline(y=0, color='red', linestyle='--', alpha=0.5)
axes[1].set_title('Daily Price Changes (%)')
axes[1].set_ylabel('Change (%)')
axes[1].grid(True, alpha=0.3)

# Rolling volatility
df['volatility'] = df['price_change'].rolling(window=30).std()
axes[2].plot(df['tanggal'], df['volatility'], color='red', linewidth=2)
axes[2].set_title('30-Day Rolling Volatility')
axes[2].set_xlabel('Date')
axes[2].set_ylabel('Volatility (%)')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Cell 6: Technical Indicators
# Calculate technical indicators
def calculate_sma(data, window):
    return data.rolling(window=window).mean()

def calculate_ema(data, window):
    return data.ewm(span=window).mean()

def calculate_rsi(data, window=14):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def calculate_bollinger_bands(data, window=20, num_std=2):
    sma = data.rolling(window=window).mean()
    std = data.rolling(window=window).std()
    upper_band = sma + (std * num_std)
    lower_band = sma - (std * num_std)
    return upper_band, lower_band, sma

# Calculate indicators
df['sma_20'] = calculate_sma(df['hargaJual'], 20)
df['sma_50'] = calculate_sma(df['hargaJual'], 50)
df['ema_12'] = calculate_ema(df['hargaJual'], 12)
df['ema_26'] = calculate_ema(df['hargaJual'], 26)
df['rsi'] = calculate_rsi(df['hargaJual'])
df['bb_upper'], df['bb_lower'], df['bb_middle'] = calculate_bollinger_bands(df['hargaJual'])

# Plot technical indicators
fig, axes = plt.subplots(3, 1, figsize=(15, 12))
fig.suptitle('Technical Indicators Analysis', fontsize=16, fontweight='bold')

# Price with moving averages
axes[0].plot(df['tanggal'], df['hargaJual'], label='Price', color='black', alpha=0.7)
axes[0].plot(df['tanggal'], df['sma_20'], label='SMA 20', color='blue')
axes[0].plot(df['tanggal'], df['sma_50'], label='SMA 50', color='red')
axes[0].plot(df['tanggal'], df['ema_12'], label='EMA 12', color='green', linestyle='--')
axes[0].set_title('Price with Moving Averages')
axes[0].set_ylabel('Price (IDR)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Bollinger Bands
axes[1].plot(df['tanggal'], df['hargaJual'], label='Price', color='black')
axes[1].plot(df['tanggal'], df['bb_upper'], label='Upper Band', color='red', alpha=0.7)
axes[1].plot(df['tanggal'], df['bb_lower'], label='Lower Band', color='red', alpha=0.7)
axes[1].plot(df['tanggal'], df['bb_middle'], label='Middle Band', color='blue', alpha=0.7)
axes[1].fill_between(df['tanggal'], df['bb_upper'], df['bb_lower'], alpha=0.1, color='gray')
axes[1].set_title('Bollinger Bands')
axes[1].set_ylabel('Price (IDR)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# RSI
axes[2].plot(df['tanggal'], df['rsi'], color='purple', linewidth=2)
axes[2].axhline(y=70, color='red', linestyle='--', alpha=0.7, label='Overbought (70)')
axes[2].axhline(y=30, color='green', linestyle='--', alpha=0.7, label='Oversold (30)')
axes[2].axhline(y=50, color='gray', linestyle='-', alpha=0.5)
axes[2].set_title('Relative Strength Index (RSI)')
axes[2].set_xlabel('Date')
axes[2].set_ylabel('RSI')
axes[2].set_ylim(0, 100)
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Cell 7: Correlation Analysis
# Calculate correlation matrix
correlation_cols = ['hargaJual', 'hargaBeli', 'spread', 'price_change', 'volatility', 
                   'sma_20', 'sma_50', 'ema_12', 'ema_26', 'rsi']

corr_matrix = df[correlation_cols].corr()

# Plot correlation heatmap
plt.figure(figsize=(12, 8))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, cmap='RdYlBu_r', center=0,
            square=True, linewidths=0.5, cbar_kws={"shrink": .8})
plt.title('Correlation Matrix - Gold Price Features', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Print strongest correlations
print("\n🔗 Strongest Correlations with Selling Price:")
price_correlations = corr_matrix['hargaJual'].abs().sort_values(ascending=False)
for feature, corr in price_correlations.items():
    if feature != 'hargaJual':
        print(f"{feature}: {corr:.3f}")

# Cell 8: Seasonality and Patterns
# Extract time features
df['year'] = df['tanggal'].dt.year
df['month'] = df['tanggal'].dt.month
df['day_of_week'] = df['tanggal'].dt.dayofweek
df['day_of_year'] = df['tanggal'].dt.dayofyear

# Monthly patterns
monthly_stats = df.groupby('month')['hargaJual'].agg(['mean', 'std']).reset_index()
monthly_stats['month_name'] = pd.to_datetime(monthly_stats['month'], format='%m').dt.month_name()

# Day of week patterns
dow_stats = df.groupby('day_of_week')['hargaJual'].agg(['mean', 'std']).reset_index()
dow_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
dow_stats['day_name'] = [dow_names[i] for i in dow_stats['day_of_week']]

# Plot seasonal patterns
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Seasonal Patterns in Gold Prices', fontsize=16, fontweight='bold')

# Monthly patterns
axes[0, 0].bar(monthly_stats['month_name'], monthly_stats['mean'], 
               color='gold', alpha=0.7, edgecolor='black')
axes[0, 0].set_title('Average Price by Month')
axes[0, 0].set_ylabel('Average Price (IDR)')
axes[0, 0].tick_params(axis='x', rotation=45)

# Day of week patterns
axes[0, 1].bar(dow_stats['day_name'], dow_stats['mean'], 
               color='silver', alpha=0.7, edgecolor='black')
axes[0, 1].set_title('Average Price by Day of Week')
axes[0, 1].set_ylabel('Average Price (IDR)')
axes[0, 1].tick_params(axis='x', rotation=45)

# Monthly volatility
axes[1, 0].bar(monthly_stats['month_name'], monthly_stats['std'], 
               color='red', alpha=0.7, edgecolor='black')
axes[1, 0].set_title('Price Volatility by Month')
axes[1, 0].set_ylabel('Standard Deviation (IDR)')
axes[1, 0].tick_params(axis='x', rotation=45)

# Price changes distribution
axes[1, 1].hist(df['price_change'].dropna(), bins=30, alpha=0.7, 
                color='blue', edgecolor='black')
axes[1, 1].axvline(x=0, color='red', linestyle='--', alpha=0.7)
axes[1, 1].set_title('Distribution of Daily Price Changes')
axes[1, 1].set_xlabel('Price Change (%)')
axes[1, 1].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

# Cell 9: Outlier Detection
from scipy import stats

# Z-score method
z_scores = np.abs(stats.zscore(df['hargaJual']))
z_outliers = df[z_scores > 3]

# IQR method
Q1 = df['hargaJual'].quantile(0.25)
Q3 = df['hargaJual'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
iqr_outliers = df[(df['hargaJual'] < lower_bound) | (df['hargaJual'] > upper_bound)]

print(f"\n🎯 Outlier Detection Results:")
print(f"Z-score outliers (|z| > 3): {len(z_outliers)} ({len(z_outliers)/len(df)*100:.1f}%)")
print(f"IQR outliers: {len(iqr_outliers)} ({len(iqr_outliers)/len(df)*100:.1f}%)")

if len(iqr_outliers) > 0:
    print(f"\nIQR Outlier dates and prices:")
    for idx, row in iqr_outliers.iterrows():
        print(f"  {row['tanggal'].date()}: {row['hargaJual']:,.0f} IDR")

# Plot outliers
plt.figure(figsize=(15, 6))
plt.plot(df['tanggal'], df['hargaJual'], label='Price', alpha=0.7)
if len(iqr_outliers) > 0:
    plt.scatter(iqr_outliers['tanggal'], iqr_outliers['hargaJual'], 
                color='red', s=50, label=f'IQR Outliers ({len(iqr_outliers)})', zorder=5)
plt.axhline(y=lower_bound, color='red', linestyle='--', alpha=0.5, label='IQR Bounds')
plt.axhline(y=upper_bound, color='red', linestyle='--', alpha=0.5)
plt.title('Gold Price with Outlier Detection', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Price (IDR)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Cell 10: Interactive Plotly Visualization
# Create interactive time series plot
fig = go.Figure()

# Add price traces
fig.add_trace(go.Scatter(
    x=df['tanggal'],
    y=df['hargaJual'],
    mode='lines',
    name='Selling Price',
    line=dict(color='gold', width=2),
    hovertemplate='Date: %{x}<br>Selling Price: %{y:,.0f} IDR<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=df['tanggal'],
    y=df['hargaBeli'],
    mode='lines',
    name='Buying Price',
    line=dict(color='silver', width=2),
    hovertemplate='Date: %{x}<br>Buying Price: %{y:,.0f} IDR<extra></extra>'
))

# Add moving averages
fig.add_trace(go.Scatter(
    x=df['tanggal'],
    y=df['sma_20'],
    mode='lines',
    name='SMA 20',
    line=dict(color='blue', width=1, dash='dash'),
    visible='legendonly'
))

fig.add_trace(go.Scatter(
    x=df['tanggal'],
    y=df['sma_50'],
    mode='lines',
    name='SMA 50',
    line=dict(color='red', width=1, dash='dash'),
    visible='legendonly'
))

# Update layout
fig.update_layout(
    title={
        'text': 'Interactive Gold Price Analysis',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 20}
    },
    xaxis_title='Date',
    yaxis_title='Price (IDR)',
    hovermode='x unified',
    height=600,
    showlegend=True,
    template='plotly_white'
)

# Add range selector
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=3, label="3m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(visible=True),
        type="date"
    )
)

fig.show()

# Cell 11: Summary Statistics and Insights
print("\n📋 COMPREHENSIVE EDA SUMMARY")
print("=" * 50)

print(f"\n📊 Dataset Characteristics:")
print(f"• Total observations: {len(df):,}")
print(f"• Date range: {df['tanggal'].min().date()} to {df['tanggal'].max().date()}")
print(f"• Time span: {(df['tanggal'].max() - df['tanggal'].min()).days} days")

print(f"\n💰 Price Statistics:")
print(f"• Selling price range: {df['hargaJual'].min():,.0f} - {df['hargaJual'].max():,.0f} IDR")
print(f"• Average selling price: {df['hargaJual'].mean():,.0f} IDR")
print(f"• Price volatility (CV): {(df['hargaJual'].std()/df['hargaJual'].mean())*100:.2f}%")

print(f"\n📈 Market Dynamics:")
print(f"• Average bid-ask spread: {df['spread'].mean():,.0f} IDR ({df['spread_pct'].mean():.2f}%)")
print(f"• Daily returns - Mean: {df['price_change'].mean():.3f}%, Std: {df['price_change'].std():.3f}%")
print(f"• Maximum single-day gain: {df['price_change'].max():.2f}%")
print(f"• Maximum single-day loss: {df['price_change'].min():.2f}%")

print(f"\n🎯 Data Quality:")
print(f"• Missing values: {df.isnull().sum().sum()}")
print(f"• Outliers (IQR method): {len(iqr_outliers)} ({len(iqr_outliers)/len(df)*100:.1f}%)")
print(f"• Negative price occurrences: {negative_jual + negative_beli}")

print(f"\n🔍 Key Insights:")
if df['price_change'].mean() > 0:
    print(f"• Overall trend: Upward (avg daily change: +{df['price_change'].mean():.3f}%)")
else:
    print(f"• Overall trend: Downward (avg daily change: {df['price_change'].mean():.3f}%)")

# Find most volatile period
if len(df) > 30:
    most_volatile_idx = df['volatility'].idxmax()
    if not pd.isna(most_volatile_idx):
        print(f"• Most volatile period: {df.loc[most_volatile_idx, 'tanggal'].date()} ({df.loc[most_volatile_idx, 'volatility']:.2f}% volatility)")

# Price momentum
recent_trend = df['price_change'].tail(7).mean()
if recent_trend > 0.1:
    print(f"• Recent momentum: Bullish (last 7 days avg: +{recent_trend:.2f}%)")
elif recent_trend < -0.1:
    print(f"• Recent momentum: Bearish (last 7 days avg: {recent_trend:.2f}%)")
else:
    print(f"• Recent momentum: Neutral (last 7 days avg: {recent_trend:.2f}%)")

print(f"\n🚀 Model Development Recommendations:")
print(f"• Sequence length: Consider 20-30 days based on autocorrelation")
print(f"• Feature engineering: Include technical indicators (RSI, moving averages)")
print(f"• Outlier handling: Apply IQR-based filtering or robust scaling")
print(f"• Validation strategy: Use time-series split to avoid data leakage")
if df['price_change'].std() > 2:
    print(f"• Volatility consideration: High volatility detected, consider ensemble methods")

print(f"\n✨ Analysis Complete! Data is ready for LSTM modeling.")